### Set path to original pyNeuroChem. Please change to your own path

In [1]:
import sys
sys.path.append('/home/olexandr/notebooks/ASE_ANI/lib')
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

In [2]:
import numpy as np
import  ase
import time
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS 

In [3]:
# Load the 
bz = read('data/bz.xyz')

In [4]:
# Set locations of all required network files
wkdir = '../ani-1x_dft_x8ens/' # Note the relative path
cnstfile = wkdir + 'rHCNO-5.2R_16-3.5A_a4-8.params' # AEV parameters
saefile  = wkdir + 'sae_linfit.dat' # Atomic shifts
nnfdir   = wkdir + '/train' # network prefix
Nn = 8 # Number of networks in the ensemble
gpu = 0 # GPU ID -- this defaults to 0

# Initilize network ensemble
aens = ensemblemolecule(cnstfile, saefile, nnfdir, Nn, gpu)

# Set ANI ensemble calculator
bz.set_calculator(ANIENS(aens))

In [5]:
start_time = time.time()
dyn = LBFGS(bz)
dyn.run(fmax=0.001)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

       Step     Time          Energy         fmax
LBFGS:    0 21:22:35    -6317.892634        0.6379
LBFGS:    1 21:22:35    -6317.901482        0.2884
LBFGS:    2 21:22:35    -6317.905798        0.0841
LBFGS:    3 21:22:35    -6317.906416        0.0420
LBFGS:    4 21:22:35    -6317.906898        0.0200
LBFGS:    5 21:22:35    -6317.906912        0.0037
LBFGS:    6 21:22:35    -6317.906911        0.0004
[ANI Total time: 0.11014962196350098 seconds]


In [6]:
# Temperature
T = 300.0

In [7]:
# We want to run MD with constant energy using the Langevin algorithm
# with a time step of 5 fs, the temperature T and the friction
# coefficient to 0.02 atomic units.
#dyn = Langevin(bz, 2 * units.fs, T * units.kB, 0.02)
dyn = NVTBerendsen(bz, 0.5 * units.fs, 300.0, taut=1.0*1000*units.fs)

In [8]:
def printenergy(a=bz):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

In [9]:
dyn.attach(printenergy, interval=100)

In [10]:
printenergy()

Energy per atom: Epot = -526.492eV  Ekin = 0.000eV (T=  0K)  Etot = -526.492eV


In [11]:
start_time = time.time()
dyn.run(2000) # Do 1ps of MD
print('[ANI Total time:', time.time() - start_time, 'seconds]')

/apps/ase/ase/md/nvtberendsen.py:66: RuntimeWarning: divide by zero encountered in double_scalars
  (self.temperature / old_temperature - 1.0) *


Energy per atom: Epot = -526.492eV  Ekin = 0.000eV (T=  0K)  Etot = -526.492eV
Energy per atom: Epot = -526.492eV  Ekin = 0.001eV (T=  7K)  Etot = -526.491eV
Energy per atom: Epot = -526.492eV  Ekin = 0.003eV (T= 24K)  Etot = -526.489eV
Energy per atom: Epot = -526.491eV  Ekin = 0.004eV (T= 32K)  Etot = -526.487eV
Energy per atom: Epot = -526.489eV  Ekin = 0.004eV (T= 31K)  Etot = -526.485eV
Energy per atom: Epot = -526.488eV  Ekin = 0.005eV (T= 38K)  Etot = -526.484eV
Energy per atom: Epot = -526.489eV  Ekin = 0.007eV (T= 54K)  Etot = -526.482eV
Energy per atom: Epot = -526.489eV  Ekin = 0.009eV (T= 68K)  Etot = -526.480eV
Energy per atom: Epot = -526.489eV  Ekin = 0.010eV (T= 81K)  Etot = -526.479eV
Energy per atom: Epot = -526.489eV  Ekin = 0.012eV (T= 91K)  Etot = -526.477eV
Energy per atom: Epot = -526.487eV  Ekin = 0.011eV (T= 83K)  Etot = -526.476eV
Energy per atom: Epot = -526.485eV  Ekin = 0.010eV (T= 76K)  Etot = -526.475eV
Energy per atom: Epot = -526.486eV  Ekin = 0.013eV (